In [1]:
import pydicom
from tqdm.auto import tqdm
from pathos.multiprocessing import ProcessingPool as Pool
from functools import partial
import pickle
import os

In [2]:
def process_xray(path):

    # Read the dicom file
    dicom = pydicom.read_file(path)
    
    meta = {
        'width': dicom.get('Columns'),
        'height': dicom.get('Rows'),
    }
    
    return meta

def resize_and_save(img_filename):
    # Preprocess and resize the x-ray
    fullpath = os.path.join(f'/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/test/', img_filename)
    meta = process_xray(fullpath)
    
    # Return the image id and meta data
    img_id = img_filename.replace('.dicom', '') 
    return (img_id, meta)

# %mkdir train
# %mkdir test

meta_data = []

# for test_train in ['train', 'test']:
for test_train in ['test']:
    imgs = os.listdir(f'/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/{test_train}/')
    
    # Process all the images in parallel
    with Pool(4) as p:
        meta_data.extend(list(tqdm(p.imap(resize_and_save, imgs), total=len(imgs))))


  0%|          | 0/3000 [00:00<?, ?it/s]

In [3]:
meta_data = dict(meta_data)
pickle.dump(meta_data, open('test_image_sizes.pickle', 'wb'))